# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770841942498                   -0.52    8.0         
  2   -2.772144771788       -2.89       -1.32    1.0    151ms
  3   -2.772170355564       -4.59       -2.48    1.0    189ms
  4   -2.772170657627       -6.52       -3.21    1.0    157ms
  5   -2.772170721880       -7.19       -3.95    2.0    180ms
  6   -2.772170722921       -8.98       -4.61    1.0    205ms
  7   -2.772170723011      -10.05       -5.17    2.0    183ms
  8   -2.772170723015      -11.40       -6.49    1.0    176ms
  9   -2.772170723015      -13.30       -6.38    2.0    197ms
 10   -2.772170723015      -14.51       -7.08    1.0    182ms
 11   -2.772170723015      -14.05       -8.11    2.0    211ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770735788973                   -0.53    9.0         
  2   -2.7

1.7735578875269047

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770705733740                   -0.53    8.0         
  2   -2.772055019468       -2.87       -1.30    1.0    210ms
  3   -2.772083004421       -4.55       -2.64    1.0    156ms
  4   -2.772083414422       -6.39       -3.89    2.0    179ms
  5   -2.772083417483       -8.51       -4.29    2.0    249ms
  6   -2.772083417804       -9.49       -5.50    1.0    161ms
  7   -2.772083417811      -11.16       -6.08    2.0    192ms
  8   -2.772083417811      -13.10       -6.81    1.0    199ms
  9   -2.772083417811      -13.85       -7.33    2.0    197ms
 10   -2.772083417811   +  -14.24       -8.42    2.0    208ms

Polarizability via ForwardDiff:       1.7725349565311803
Polarizability via finite difference: 1.7735578875269047
